In [ ]:
import os
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [ ]:
data = sparkSession.read.parquet("/data/sample264")
dataforNext=sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")
dataforNext=dataforNext.select('userId',  \
                         dataforNext.trackId.alias('trackId1'), \
                         dataforNext.timestamp.alias('timestamp1'))


## Normalization could be done by next function

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, sum

def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    topsDF = df.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= n) \
        .drop(col("row_number")) 
        
    tmpDF = topsDF.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias("sum_" + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, "inner") \
        .withColumn("norm_" + field, col(field) / col("sum_" + field)) \
        .cache()

    return normalizedDF

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, rank,when

dataJoined = data.join(dataforNext, data.userId == dataforNext.userId )\
                       .filter(data.trackId!=dataforNext.trackId1) \
                       .filter((dataforNext.timestamp1-data.timestamp<=420) & (dataforNext.timestamp1-data.timestamp>0) ) \
                    .select(data.userId,  \
                         data.trackId.alias('trackId1'), \
                         dataforNext.trackId1.alias('trackId2')) \
                .withColumn("id", when((col('trackId2')>=col('trackId1')),col('trackId1')).otherwise(col('trackId2'))) \
                .withColumn("id2", when((col('trackId1')<=col('trackId2')),col('trackId2')).otherwise(col('trackId1'))) 
trackData = dataJoined.groupBy('id', 'id2').count()
trackDataNorm = norm(trackData, "id","id2", "count", 40)
window = Window.orderBy(col("norm_count").desc())
trackList = trackDataNorm.withColumn("position", rank().over(window)) \
                         .filter(col("position") <= 50)\
                         .sort(["norm_count","id","id2"],ascending=[0,1,1])\
                        .select(col("id"), col("id2"))\
                          .take(40)


In [ ]:
for val in trackList:
    print "%s %s" % val